# Data preprocessing for the Twitter dataset

In [9]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
from langdetect import detect, DetectorFactory, LangDetectException
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch
from tqdm import tqdm
import nltk
nltk.download("punkt")
nltk.download("wordnet")

import pandas as pd
import re
import string
from langdetect import detect, LangDetectException
from tqdm.auto import tqdm
import spacy
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paull\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\paull\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paull\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
musk_twitter_data = pd.read_csv(os.path.join('raw', 'all_musk_posts.csv'),parse_dates=["createdAt"])

start_date = "2015-01-01"
end_date = musk_twitter_data["createdAt"].max()

# end_date = musk_twitter_data["createdAt"].max()

musk_twitter_data = musk_twitter_data[musk_twitter_data["createdAt"] > start_date]
musk_twitter_data = musk_twitter_data[musk_twitter_data["createdAt"] < end_date]

musk_twitter_data["isRetweet"] = musk_twitter_data["isRetweet"].astype(str).str.lower()
musk_twitter_data["possiblySensitive"] = musk_twitter_data["possiblySensitive"].astype(str).str.lower()
musk_twitter_data["fullText"] = musk_twitter_data["fullText"].astype(str)
musk_twitter_data["date"] = musk_twitter_data["createdAt"].dt.date

musk_twitter_data_all = musk_twitter_data.copy()
musk_twitter_data_nlp = musk_twitter_data.copy()

musk_twitter_data_nlp.shape, musk_twitter_data_all.shape, musk_twitter_data.shape

C:\Users\paull\AppData\Local\Temp\ipykernel_24448\52893527.py:1: DtypeWarning: Columns (11,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  musk_twitter_data = pd.read_csv(os.path.join('raw', 'all_musk_posts.csv'),parse_dates=["createdAt"])


((54023, 25), (54023, 25), (54023, 25))

In [10]:
# --- Setup ---
tqdm.pandas()
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
stop_words = set(stopwords.words('english'))

# --- Helpers ---
def safe_detect(text):
    try:
        text = str(text).strip()
        if len(text) < 10:
            return "unknown"
        return detect(text)
    except LangDetectException:
        return "unknown"

def clean_basic(text):
    text = str(text)
    text = re.sub(r"http\S+|www\S+|@\w+|#|RT", "", text)
    return text.strip()

def clean_for_topic(text):
    text = clean_basic(text).lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\d+", "", text)
    return text

def preprocess_lemmatized(text):
    text = clean_for_topic(text)
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words])

# --- NLP DataFrame creation ---
musk_twitter_data_nlp = musk_twitter_data.copy()

# Filter for valid original texts
musk_twitter_data_nlp = musk_twitter_data_nlp[
    (musk_twitter_data_nlp["isRetweet"] != "true") &
    (musk_twitter_data_nlp["fullText"].str.strip() != "")
]

# Language detection
musk_twitter_data_nlp["language"] = musk_twitter_data_nlp["fullText"].progress_apply(safe_detect)
musk_twitter_data_nlp = musk_twitter_data_nlp[musk_twitter_data_nlp["language"] == "en"]

# Text variations
musk_twitter_data_nlp["text_raw"] = musk_twitter_data_nlp["fullText"].progress_apply(clean_basic)
musk_twitter_data_nlp["text_lemmatized"] = musk_twitter_data_nlp["fullText"].progress_apply(preprocess_lemmatized)

  0%|          | 0/52939 [00:00<?, ?it/s]

  0%|          | 0/41819 [00:00<?, ?it/s]

  0%|          | 0/41819 [00:00<?, ?it/s]

In [12]:
print("Number of tweets after filtering: ", musk_twitter_data_nlp.shape[0])
print("Number of tweets before filtering: ", musk_twitter_data.shape[0])

display(musk_twitter_data_nlp.head()) 


Number of tweets after filtering:  41819
Number of tweets before filtering:  54023


,id,url,twitterUrl,fullText,retweetCount,replyCount,likeCount,quoteCount,viewCount,createdAt,...,isQuote,isConversationControlled,possiblySensitive,quoteId,quote,retweet,date,language,text_raw,text_lemmatized
0,1655159652990976000,https://x.com/elonmusk/status/1655159652990976000,https://twitter.com/elonmusk/status/1655159652...,RT @einarvollset: I read @paulg’s “How to Mak...,NaN,NaN,NaN,NaN,NaN,2023-05-07 10:36:27+00:00,...,NaN,NaN,nan,NaN,NaN,NaN,2023-05-07,en,: I read ’s “How to Make Wealth” in Hackers &...,I read make wealth hacker amp painter I mid tw...
2,1623774484795920384,https://x.com/elonmusk/status/1623774484795920384,https://twitter.com/elonmusk/status/1623774484...,RT @BillyM2k: dude bookmarks are an awesome tw...,NaN,NaN,NaN,NaN,NaN,2023-02-09 20:03:00+00:00,...,NaN,NaN,nan,NaN,NaN,NaN,2023-02-09,en,: dude bookmarks are an awesome twitter featur...,dude bookmark awesome twitter feature especial...
3,1656900119202254854,https://x.com/elonmusk/status/1656900119202254854,https://twitter.com/elonmusk/status/1656900119...,Event Horizon Balance Beam,NaN,NaN,NaN,NaN,NaN,2023-05-12 05:52:26+00:00,...,NaN,NaN,nan,NaN,NaN,NaN,2023-05-12,en,Event Horizon Balance Beam,event horizon balance beam
4,1616531874763116544,https://x.com/elonmusk/status/1616531874763116544,https://twitter.com/elonmusk/status/1616531874...,RT @BillyM2k: @elonmusk oh that’s actually pre...,NaN,NaN,NaN,NaN,NaN,2023-01-20 20:23:27+00:00,...,NaN,NaN,nan,NaN,NaN,NaN,2023-01-20,en,: oh that’s actually pretty cool!,oh actually pretty cool
5,1608035930983604230,https://x.com/elonmusk/status/1608035930983604230,https://twitter.com/elonmusk/status/1608035930...,RT @SpaceX: Watch Falcon 9 launch 54 Starlink ...,NaN,NaN,NaN,NaN,NaN,2022-12-28 09:43:36+00:00,...,NaN,NaN,nan,NaN,NaN,NaN,2022-12-28,en,: Watch Falcon 9 launch 54 Starlink satellites...,watch falcon launch starlink satellite orbit


In [ ]:
# To csv
musk_twitter_data_nlp.to_csv(os.path.join('cleaned', 'musk_twitter_data_nlp.csv'), index=False)
musk_twitter_data_all.to_csv(os.path.join('cleaned', 'musk_twitter_data_all.csv'), index=False)